# Chatbot con voz

### Ana Juárez
### Mario Mora

Responde al consultársele la información un jugador de fútbol, o bien por el resultado de un partido específico, por ejemplo "Resultado entre Barcelona y Valencia", o "Marcador de Getafe contra Barcelona".

Iniciamos importando las librerías requeridas para este proyecto

In [1]:
import speech_recognition as sr

import json

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import cess_esp
from nltk.tag.hmm import HiddenMarkovModelTagger, HiddenMarkovModelTrainer

from nltk.chunk.regexp import *

from urllib.request import urlopen

import requests, pyttsx3


Definimos un corpus con los nombres de los equipos de las principales ligas europeas

### Definimos un HiddenMarkovModelTagger

Lo entrenamos con el corpus, y posteriormente con el cess_esp para detectar palabras en español

In [2]:
hmm = HiddenMarkovModelTagger.train(cess_esp.tagged_sents())

Creamos una función para etiquetado de palabras

In [3]:
def taggear(_frase):
    return hmm.tag(word_tokenize(_frase))

Creamos las reglas gramáticas para la detección de un marcador

In [4]:
reglas_marcador = """
Equipo: { <np.*> }
Equipo: { <Equipo> <np.*> }
Partido: { <Equipo>  <sp.*> <Equipo> | <Equipo> <cc.*> <Equipo> }
"""
parser_marcador = nltk.RegexpParser(reglas_marcador)


In [5]:
reglas_jugador = """
Jugador: { <di.*> <nc.*> }
"""
parser_jugador = nltk.RegexpParser(reglas_jugador)


Se obtiene la información del árbol para a través de sus hojas determinar los equipos y consultar el marcador a través de la API

In [6]:
def obtener_Info(_tree, tipo):
    resultado = ''
    for nodo in _tree:
        if(type(nodo) != tuple):
            #Si encuentra un nodo de tipo partido
            if(nodo.label() == 'Partido'):
                for elemento in nodo:
                    if(type(elemento) != tuple):
                        if(elemento.label() == 'Equipo'):
                            resultado += ' '.join([ hoja[0] for hoja in elemento.leaves()]) + ' '
            if(nodo.label() == 'Jugador'):
                resultado += ' '.join([ hoja[0] for hoja in nodo.leaves()]) + ' '

    #Vamos a formar el parámetro que se va a enviar a la API
    if(tipo == 'Partido'):
        return resultado.strip().replace(' ', '_vs_')
    else: 
        return resultado.strip().replace(' ', '_')

Función para hacer la consulta de un marcador a la API

In [7]:
def obtener_Marcador(_frase):
    url = 'https://www.thesportsdb.com/api/v1/json/3/searchevents.php?e=' + obtener_Info(_frase, 'Partido')    
    data = requests.get(url)
    evento = json.loads(data.text)
    if(evento['event'][0]['intHomeScore'] == None):
        return 'Este encuentro no se ha disputado'
    return f"El resultado del último encuentro de {evento['event'][0]['strHomeTeam']} y {evento['event'][0]['strAwayTeam']} fue de {evento['event'][0]['intHomeScore']} a {evento['event'][0]['intAwayScore']}"

Función para hacer la consulta de un jugador a la API

In [8]:
def obtener_Jugador(_frase):
    url = "https://thesportsdb.com/api/v1/json/3/searchplayers.php?p=" + _frase
    data = requests.get(url)
    jugador = json.loads(data.text)
    if(jugador['player'][0]['idPlayer'] == None):
        return 'Jugador no encontrado'
    return f"{jugador['player'][0]['strPlayer']} es un jugador de {jugador['player'][0]['strBirthLocation']}, actualmente juega en el {jugador['player'][0]['strTeam']} y la selección de {jugador['player'][0]['strNationality']}"    

Función para capturar la consulta a través del micrófono

In [9]:
def listen():
    mic = sr.Microphone()
    with mic as source:
        instance = sr.Recognizer()
        audio = instance.listen(source)
        transcript = instance.recognize_google(audio, language='es-ES', show_all=True)
        print(transcript['alternative'][0]['transcript'])
        return transcript['alternative'][0]['transcript']

Función para dar una respuesta por audio

In [10]:
engine = pyttsx3.init()
def speak(_text, _voice = 0):
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[_voice].id)
    engine.setProperty('rate', 130)
    engine.say(_text)
    engine.runAndWait()

In [11]:
texto = listen()

resultado = ''

if(('resultado' in texto) or ('marcador' in texto)):
    frase = parser_marcador.parse(taggear(texto))
    resultado = obtener_Marcador(frase)
else:
    resultado = obtener_Jugador(obtener_Info(parser_jugador.parse(taggear(texto)), 'Jugador'))

print(resultado)
speak(resultado, 3) # Se utiliza la voz número 3 instalada en nuestro ordenador, correspondiente a español de México, modificar si no se posee


quién es Luis Suárez
Luis Suarez es un jugador de Salto, Uruguay, actualmente juega en el Inter Miami y la selección de Uruguay
